# 0.1. imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

# 0.2. config

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.3. load `pylcaio`

In [3]:
sys.path.append(str(Path(Path.home(), config['path_dir_repo_pylcaio'])))
import pylcaio 

# 0.4. file paths
## 0.4.1. names

In [4]:
%%capture
print(ecoinvent_version_name := config['ecoinvent_version_name'])

## 0.4.2. directories

In [5]:
%%capture
print(path_dir_data := Path(Path.home(), config['path_dir_data']))
print(path_dir_data_raw := Path(path_dir_data, config['path_dir_data_raw']))
print(path_dir_data_processed := Path(path_dir_data, config['path_dir_data_processed']))
print(path_dir_processing_logs := Path(path_dir_data_processed, config['path_dir_processing_logs']))
# ecoinvent
print(path_dir_ecoinvent := Path(path_dir_data_raw, ecoinvent_version_name))

## 0.4.3. files

In [6]:
%%capture
print(path_file_ecoinvent_characterization := Path(path_dir_data_raw, config['ecoinvent_characterization_file']))
print(path_file_e2m_class_object := Path(path_dir_data_processed, config['e2m_class_instance']))

# 1.1. load `Ecoinvent`
## 1.1.1 read ecoinvent and write to pickle
⏳ ~12min

In [7]:
parser = e2m.Ecospold2Matrix(
    sys_dir = str(path_dir_ecoinvent), # passing a Posix Path object breaks the functionality
    characterisation_file = str(path_file_ecoinvent_characterization), # passing a Posix Path object breaks the functionality
    project_name = ecoinvent_version_name,
    out_dir = path_dir_data_processed,
    positive_waste = False,
    nan2null = True
)
parser.save_interm = False
parser.prefer_pickles = True

2023-01-27 09:36:15,066 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Ecospold2Matrix Processing
2023-01-27 09:36:15,114 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Current git commit: 3b4a43c4f1ac4bf13b502dfe329f475216be5079
2023-01-27 09:36:15,115 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Project name: ecoinvent_3.5_cutoff_ecoSpold02
2023-01-27 09:36:15,115 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Unit process and Master data directory: /Users/michaelweinold/data/data_raw/ecoinvent_3.5_cutoff_ecoSpold02
2023-01-27 09:36:15,115 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Data saved in: /Users/michaelweinold/data/data_processed
2023-01-27 09:36:15,116 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Replace Not-a-Number instances with 0.0 in all matrices
2023-01-27 09:36:15,116 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Pickle intermediate results to files
2023-01-27 09:36:15,116 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Order processes based on: ISIC, activityName
2023-01-27 09:36:15,117 - e

In [8]:
parser.ecospold_to_Leontief(
    fileformats = 'Pandas',
    with_absolute_flows = True
)

2023-01-27 09:36:15,243 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Products extracted from IntermediateExchanges.xml with SHA-1 of b2c87a5bf5982a60515a6e1160e43c620a218369
2023-01-27 09:36:28,859 - ecoinvent_3.5_cutoff_ecoSpold02 - WARNING - Removed 1148 duplicate rows from activity_list, see duplicate_activity_list.csv.
2023-01-27 09:36:28,917 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Activities extracted from ActivityIndex.xml with SHA-1 of 3ac94e9826a9a031ff2e0bfbdceeecaeb72a9117
2023-01-27 09:36:28,939 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Processing 16022 files in /Users/michaelweinold/data/data_raw/ecoinvent_3.5_cutoff_ecoSpold02/datasets
2023-01-27 09:37:24,130 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Processing 16022 files - this may take a while ...
2023-01-27 09:38:39,071 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Elementary flows extracted from ElementaryExchanges.xml with SHA-1 of 0caa74a71870c1432557a91516a6da63d319b594
2023-01-27 09:38:39,091 - ecoinvent_3.5_cutof

starting characterisation


2023-01-27 09:39:05,546 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Will use column 7, named CF_35, for characterisation factors
2023-01-27 09:39:05,547 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Starting characterisation matching
2023-01-27 09:39:09,400 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Characterisation matching done. C matrix created
2023-01-27 09:39:09,444 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Starting to export to file
2023-01-27 09:39:09,444 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - about to write to file
2023-01-27 09:42:42,003 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Final, symmetric, normalized matrices saved in /Users/michaelweinold/data/data_processed/ecoinvent_3.5_cutoff_ecoSpold02Pandas_symmNorm.gz.pickle with SHA-1 of 83eab01054632e2c92dcb25d4845752dc50d6065
2023-01-27 09:46:15,208 - ecoinvent_3.5_cutoff_ecoSpold02 - INFO - Final, symmetric, scaled-up flow matrices saved in /Users/michaelweinold/data/data_processed/ecoinvent_3.5_cutoff_ecoSpold02Pandas_symm

## 1.1.2 remove all files other than the pickled dictionaries of `pd.DataFrame`s

In [9]:
Path.unlink(Path.cwd() / (ecoinvent_version_name + '_characterisation.db'), missing_ok = True)
Path.unlink(Path.cwd() / 'C_long', missing_ok = True)

# 2. investigation

In [10]:
parser._prop_dict

{'3a0af1d6-04c3-41c6-a3da-92c4f61e0eaa': 'dry mass [kg]',
 '67f102e2-9cb6-4d20-aa16-bf74d8a03326': 'wet mass [kg]'}

In [12]:
parser.products

,productName,unitName,productId,unitId,cpc,dry mass [kg],wet mass [kg]
42761d87-05d9-4877-b21e-001ecf0c747d,"trawler maintenance, steel",kg,42761d87-05d9-4877-b21e-001ecf0c747d,487df68b-4994-4027-8fdc-a4dc298257b7,8729: Maintenance and repair services of other...,NaN,NaN
c4f00122-eccc-4c3c-8068-0022aa7bf13b,"kraft paper, unbleached",kg,c4f00122-eccc-4c3c-8068-0022aa7bf13b,487df68b-4994-4027-8fdc-a4dc298257b7,"321: Pulp, paper and paperboard",0.945,1
c7bd9b85-63d3-481e-b51b-00435cbd54e8,"transport, passenger, motor scooter",person*km,c7bd9b85-63d3-481e-b51b-00435cbd54e8,0e6f38bc-51be-4382-802d-1f389a88e589,64119: Other land transportation services of p...,NaN,NaN
86247d21-f010-4ce3-a76d-0050837ddfc0,"green manure, Swiss integrated production, unt...",ha,86247d21-f010-4ce3-a76d-0050837ddfc0,86bbe475-8a8f-44d8-914c-e398787e7121,34659: Fertilizers n.e.c.,NaN,NaN
915500af-3702-4039-8ea7-005a5f84cd2d,bentonite quarry infrastructure,unit,915500af-3702-4039-8ea7-005a5f84cd2d,5b972631-34e3-4db7-a615-f6931770a0cb,53269: Other constructions for manufacturing,NaN,NaN
...,...,...,...,...,...,...,...
2677ef49-b019-4da3-bc8e-ffa396bad75f,"refrigeration machine, R134a as refrigerant",unit,2677ef49-b019-4da3-bc8e-ffa396bad75f,5b972631-34e3-4db7-a615-f6931770a0cb,43913: Refrigerating and freezing equipment an...,NaN,NaN
b397787f-c69d-45a6-876c-ffc8a6be4fc6,"section bar extrusion, aluminium",kg,b397787f-c69d-45a6-876c-ffc8a6be4fc6,487df68b-4994-4027-8fdc-a4dc298257b7,4153: Semi-finished products of aluminium or a...,NaN,NaN
1e27a902-25f3-449e-bf0f-ffe5192c0c81,"furnace, logs, softwood storage area, 100kW",unit,1e27a902-25f3-449e-bf0f-ffe5192c0c81,5b972631-34e3-4db7-a615-f6931770a0cb,43420: Industrial or laboratory furnaces and o...,NaN,NaN
a7f92ffa-a459-4c1c-ae8b-ffefe3804099,ammonium bicarbonate,kg,a7f92ffa-a459-4c1c-ae8b-ffefe3804099,487df68b-4994-4027-8fdc-a4dc298257b7,34240: Phosphates of triammonium; salts and pe...,1,1


In [17]:
parser.activities

,activityId,activityNameId,activityType,startDate,endDate,activityName
0fcbe58d-60b6-46d9-86bf-183d14732a4d,0fcbe58d-60b6-46d9-86bf-183d14732a4d,06ffc039-f73b-440c-a5a0-8dc7393a5e71,0,1997-01-01,2001-12-31,concrete block production
52f51150-6305-4bfd-ac84-418cf0e1274d,52f51150-6305-4bfd-ac84-418cf0e1274d,d785cf26-f91b-404c-ac0a-7b077b69f677,0,1997-01-01,2001-12-31,"gravel production, crushed"
6c46a126-ed7d-468d-8a93-8e8ccddc1b89,6c46a126-ed7d-468d-8a93-8e8ccddc1b89,d785cf26-f91b-404c-ac0a-7b077b69f677,0,1997-01-01,2001-12-31,"gravel production, crushed"
e57c0d07-a886-484d-89e0-677f67dc34c6,e57c0d07-a886-484d-89e0-677f67dc34c6,06ffc039-f73b-440c-a5a0-8dc7393a5e71,0,1997-01-01,2001-12-31,concrete block production
1da7c9e9-2ba0-46ac-ac6b-7a0a2b99d447,1da7c9e9-2ba0-46ac-ac6b-7a0a2b99d447,36cdf17f-b080-4f6c-a864-43ca370de943,0,1995-01-01,2000-12-31,autoclaved aerated concrete block production
...,...,...,...,...,...,...
e193a310-716b-4f60-b4e3-8a4e2f02b407,e193a310-716b-4f60-b4e3-8a4e2f02b407,3719a1da-895e-4df0-a872-c6d7d6020535,4,2018-01-01,2018-12-31,"electricity, high voltage, production mix"
a785d216-e447-4c14-80a3-d880baa438d7,a785d216-e447-4c14-80a3-d880baa438d7,3719a1da-895e-4df0-a872-c6d7d6020535,4,2018-01-01,2018-12-31,"electricity, high voltage, production mix"
c979830a-7c76-47a8-9e2c-9e769de1191c,c979830a-7c76-47a8-9e2c-9e769de1191c,3719a1da-895e-4df0-a872-c6d7d6020535,4,2018-01-01,2018-12-31,"electricity, high voltage, production mix"
9a3e7ddc-7da1-48f7-a51f-df0a454ecb1a,9a3e7ddc-7da1-48f7-a51f-df0a454ecb1a,6f877aae-5a06-4b00-9b4c-b74f8333dbc0,4,2018-01-01,2018-12-31,"electricity, high voltage, aluminium industry,..."


In [10]:
parser.inflows

,fileId,sourceActivityId,productId,amount,row_index
027d892b-34db-4c3f-9f33-934218f23871_d3392d17-8ac2-4a74-b8e8-c1b2e6784d3e,027d892b-34db-4c3f-9f33-934218f23871_d3392d17-...,a54f640d-c524-5f2c-997b-47ccc2809629,51c7e57a-00b7-4412-b56c-94c17b211ee7,4.000000e-01,a54f640d-c524-5f2c-997b-47ccc2809629_51c7e57a-...
027d892b-34db-4c3f-9f33-934218f23871_d3392d17-8ac2-4a74-b8e8-c1b2e6784d3e,027d892b-34db-4c3f-9f33-934218f23871_d3392d17-...,d13adcc8-385e-495b-a40d-163b97e0d361,96f36f31-8f6a-48f8-921f-1d781b7a545f,6.000000e-01,d13adcc8-385e-495b-a40d-163b97e0d361_96f36f31-...
ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-289e-4246-b875-6aabe1a7ad79,ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-...,51961800-d8b1-4940-a3fc-c23610368751,e7a3abb3-289e-4246-b875-6aabe1a7ad79,1.446138e-01,51961800-d8b1-4940-a3fc-c23610368751_e7a3abb3-...
ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-289e-4246-b875-6aabe1a7ad79,ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-...,99ac0aa2-99ac-4127-92e1-30980adee9dc,e7a3abb3-289e-4246-b875-6aabe1a7ad79,1.446138e-01,99ac0aa2-99ac-4127-92e1-30980adee9dc_e7a3abb3-...
ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-289e-4246-b875-6aabe1a7ad79,ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-...,5d9c46ce-24b2-439f-882c-b0ee7373b55e,e7a3abb3-289e-4246-b875-6aabe1a7ad79,3.553862e-01,5d9c46ce-24b2-439f-882c-b0ee7373b55e_e7a3abb3-...
...,...,...,...,...,...
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,73f22ac3-c760-4950-a1b7-88399ef0c335,1125e767-7b5d-442e-81d6-9b0d3e1919ac,2.770000e+00,73f22ac3-c760-4950-a1b7-88399ef0c335_1125e767-...
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,7c56c9b2-a02b-46e7-b0c9-917cd18b1ff8,759b89bd-3aa6-42ad-b767-5bb9ef5d331d,2.580000e-02,7c56c9b2-a02b-46e7-b0c9-917cd18b1ff8_759b89bd-...
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,6794c72d-c03c-41f8-a4fb-0851876e18dc,3d0fe4e0-eac9-4704-b3fd-09b8594d0fbe,4.000000e-10,6794c72d-c03c-41f8-a4fb-0851876e18dc_3d0fe4e0-...
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,8c3e7515-21f1-4f5a-b2f3-d6fc286e08f1,27da8130-82ba-485c-a800-b89efdcb0491,-4.920000e-03,8c3e7515-21f1-4f5a-b2f3-d6fc286e08f1_27da8130-...


In [21]:
parser.outflows

,fileId,productId,amount,productionVolume,outputGroup
027d892b-34db-4c3f-9f33-934218f23871_d3392d17-8ac2-4a74-b8e8-c1b2e6784d3e,21084_027d892b-34db-4c3f-9f33-934218f23871_d33...,d3392d17-8ac2-4a74-b8e8-c1b2e6784d3e,1.0,1.322369e+00,0
ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a3abb3-289e-4246-b875-6aabe1a7ad79,22385_ee600b5b-adbf-44f3-ab2e-17803c17e3f6_e7a...,e7a3abb3-289e-4246-b875-6aabe1a7ad79,1.0,1.642526e+11,0
1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-f32b-4591-901c-55395db5c132,19393_1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c...,66c93e71-f32b-4591-901c-55395db5c132,1.0,1.855062e+08,0
1244d764-41fe-497f-95c4-3d0238344c29_013d2a59-5d91-4d06-acfc-4ea125843c19,28675_1244d764-41fe-497f-95c4-3d0238344c29_013...,013d2a59-5d91-4d06-acfc-4ea125843c19,1.0,2.677631e+00,0
04dde023-6a1e-4cc4-b5a9-170488161a2c_96ca17ef-cd4b-410f-b50e-d00d58dd8b94,24464_04dde023-6a1e-4cc4-b5a9-170488161a2c_96c...,96ca17ef-cd4b-410f-b50e-d00d58dd8b94,1.0,9.272109e+06,0
...,...,...,...,...,...
c2677311-5b0a-41df-8f55-ed3a75762093_759b89bd-3aa6-42ad-b767-5bb9ef5d331d,22919_c2677311-5b0a-41df-8f55-ed3a75762093_759...,759b89bd-3aa6-42ad-b767-5bb9ef5d331d,1.0,3.593237e+09,0
d8b19ec9-8135-4312-8412-cb42a76ac65c_f372d4c2-7310-4fb9-8bb6-bee214fd08d1,21759_d8b19ec9-8135-4312-8412-cb42a76ac65c_f37...,f372d4c2-7310-4fb9-8bb6-bee214fd08d1,1.0,1.878606e+04,0
6b2aa22c-ef64-497a-ad55-aad5e1d163b0_d69294d7-8d64-4915-a896-9996a014c410,30971_6b2aa22c-ef64-497a-ad55-aad5e1d163b0_d69...,d69294d7-8d64-4915-a896-9996a014c410,1.0,8.333300e-02,0
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,25725_5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf8...,cf82552d-9740-4f5c-8df1-844cc9486547,1.0,1.322369e+00,0


In [19]:
parser.elementary_flows

,fileId,elementaryExchangeId,amount
1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-f32b-4591-901c-55395db5c132,1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-...,c0dd7ccd-9e7a-42b3-b899-dfd18c2150ca,3.790000e+00
1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-f32b-4591-901c-55395db5c132,1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-...,c14ea750-4a9f-41fa-bcc1-4a1d84466f43,4.496000e-03
1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-f32b-4591-901c-55395db5c132,1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-...,8c1494a5-4987-4715-aa2d-1908c495f4eb,4.500000e+01
1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-f32b-4591-901c-55395db5c132,1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-...,2c126bcc-bb63-4d63-bd72-f02a1e616809,2.810000e-05
1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-f32b-4591-901c-55395db5c132,1b845e0d-cce6-4c43-a94f-ed6b2e9f0631_66c93e71-...,2404b41a-2eed-4e9d-8ab6-783946fdf5d6,4.500000e+01
...,...,...,...
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,960c0f37-f34c-4fc1-b77c-22d8b35fd8d5,3.111111e-05
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,97e498ec-f323-4ec6-bcc0-d8a4c853bae3,7.710000e-08
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,175baa64-d985-4c5e-84ef-67cc3a1cf952,8.200000e-04
5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-9740-4f5c-8df1-844cc9486547,5c0ae70f-8889-4fa2-bcdf-f6387417f06c_cf82552d-...,831f249e-53f2-49cf-a93c-7cee105f048e,6.000000e-03


In [22]:
parser.STR

,id,name,unit,cas,comp,subcomp
id,,,,,,
d21da01e-f96f-4db5-9746-7b70db8a1f2c,d21da01e-f96f-4db5-9746-7b70db8a1f2c,"1,4-Butanediol",kg,000110-63-4,air,"low population density, long-term"
90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,"1,4-Butanediol",kg,000110-63-4,air,lower stratosphere + upper troposphere
83bafcf1-2f2e-4a32-89a0-f1f16ca10626,83bafcf1-2f2e-4a32-89a0-f1f16ca10626,"1,4-Butanediol",kg,000110-63-4,air,non-urban air or from high stacks
09db39be-d9a6-4fc3-8d25-1f80b23e9131,09db39be-d9a6-4fc3-8d25-1f80b23e9131,"1,4-Butanediol",kg,000110-63-4,air,unspecified
38a622c6-f086-4763-a952-7c6b3b1c42ba,38a622c6-f086-4763-a952-7c6b3b1c42ba,"1,4-Butanediol",kg,000110-63-4,air,urban air close to ground
...,...,...,...,...,...,...
e9eb024b-6fba-4dce-b280-9415b2ba4695,e9eb024b-6fba-4dce-b280-9415b2ba4695,t-Butylamine,kg,000075-64-9,water,ground-
d0aaa60f-40d9-4c12-a486-954333aa549f,d0aaa60f-40d9-4c12-a486-954333aa549f,t-Butylamine,kg,000075-64-9,water,"ground-, long-term"
cc415ec5-ef9b-4239-b423-32906818e457,cc415ec5-ef9b-4239-b423-32906818e457,t-Butylamine,kg,000075-64-9,water,ocean


In [23]:
parser.PRO

,activityId,productId,activityName,ISIC,price,priceUnit,EcoSpoldCategory,geography,technologyLevel,macroEconomicScenario,...,productName,unitName,cpc,dry mass [kg]_y,wet mass [kg]_y,activityNameId,activityType,startDate,endDate,activityName_duplicate
index,,,,,,,,,,,,,,,,,,,,,
a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,a96cb241-a4a9-4980-a16a-ba4b6a80175e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CA-QC,Current,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production"
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production"
d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,d1c3b8ad-58b6-4ea4-8877-a51b356556bc,281fc4f0-c05d-410a-a784-06e3508e78e6,"barley grain, feed production, Swiss integrate...","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CH,Current,Business-as-Usual,...,"barley grain, feed, Swiss integrated production",kg,23319: Preparations used in animal feeding n.e.c.,0.85,1,938a1c4e-41f2-40ea-9bf1-9f09c2437798,0,1996-01-01,2018-12-31,"barley grain, feed production, Swiss integrate..."
92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,92068396-88c7-45ed-9008-622008a299f3,0d860eb4-1a25-41b4-a821-81f5726d86e5,"barley grain, feed production, organic","0111:Growing of cereals (except rice), legumin...",0.159,EUR2005,agricultural means of production/feed,CH,Current,Business-as-Usual,...,"barley grain, feed, organic",kg,23319: Preparations used in animal feeding n.e.c.,0.85,1,6a49a0c6-c114-4471-b6c4-8aa4006e4b0e,0,1996-01-01,2018-12-31,"barley grain, feed production, organic"
35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c,0d860eb4-1a25-41b4-a821-81f5726d86e5,"barley grain, feed production, organic","0111:Growing of cereals (except rice), legumin...",0.159,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,"barley grain, feed, organic",kg,23319: Preparations used in animal feeding n.e.c.,0.85,1,6a49a0c6-c114-4471-b6c4-8aa4006e4b0e,0,1996-01-01,2018-12-31,"barley grain, feed production, organic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,31e6ab74-141d-49f4-9e75-5974bd62742b,b3184435-d00c-5713-823d-5ff6741bfbbc,"waste polyethylene terephthalate, for recyclin...",NaN,NaN,NaN,NaN,GLO,Current,Business-as-Usual,...,"waste polyethylene terephthalate, for recyclin...",kg,"39270: Waste, parings and scrap of plastics",NaN,NaN,712ffc38-2281-4fd2-a84d-089678e23bd3,0,2018-01-01,2018-12-31,"waste polyethylene terephthalate, for recyclin..."
b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,b50b032c-8bf0-41a4-bb7a-ce5e12cabf01,d13ae29b-7d6a-55d2-8380-86343a80fed8,"waste polyethylene terephthalate, for recyclin...",NaN,NaN,NaN,NaN,GLO,Current,Business-as-Usual,...,"waste polyethylene terephthalate, for recyclin...",kg,"39270: Waste, parings and scrap of plastics",NaN,NaN,27dcd766-bc63-4a7c-9fdf-ffc55e0575b2,0,2018-01-01,2018-12-31,"waste polyethylene terephthalate, for recyclin..."
307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,307b681d-c6bc-44cb-ab3e-a9067a4e1

In [12]:
parser.A

index,a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,8292bb2c-903f-400e-861e-fa427f39e262_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,60b530e4-1f8c-44bc-aa12-5cd4d2187dae_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,558a3696-1009-44a7-8753-da7217f46c77_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,9d4929b6-4d6b-41ed-ba7e-0abd6c974867_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,c8c5da0c-49af-43d7-82a8-5d46d66f495c_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,...,bd1d1d91-fc01-4b4c-ad58-858c00bab907_8d7c013a-5aca-44d7-8fc4-193844d2953a,5528611b-1e96-4e7e-b5ea-a103534074ba_0baac448-32a1-4b9f-afec-69a9b9b203ac,c60bcfeb-d21f-4d2e-915b-a5bb52a1929f_f70c1877-6db5-4024-b260-69f50314b40c,3c138178-0520-4950-af40-23fce8501056_fd27af60-7b68-4549-bf35-022eacd74f93,d448c7f9-cd47-4456-8a32-0d0e8e024c14_ad38bbd2-e41c-4509-9dc3-c5c708a36606,31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,73c9e29c-885e-490e-95f3-087227443758_101e55f2-2da0-528b-96fd-21687e242c1c,91ed532a-08f2-4fdd-b5e9-28bb2afd1336_451550a8-d9e6-4396-9f9a-97e752111228
index,,,,,,,,,,,,,,,,,,,,,
a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
parser.F

index,a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,8292bb2c-903f-400e-861e-fa427f39e262_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,60b530e4-1f8c-44bc-aa12-5cd4d2187dae_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,558a3696-1009-44a7-8753-da7217f46c77_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,9d4929b6-4d6b-41ed-ba7e-0abd6c974867_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,c8c5da0c-49af-43d7-82a8-5d46d66f495c_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,...,bd1d1d91-fc01-4b4c-ad58-858c00bab907_8d7c013a-5aca-44d7-8fc4-193844d2953a,5528611b-1e96-4e7e-b5ea-a103534074ba_0baac448-32a1-4b9f-afec-69a9b9b203ac,c60bcfeb-d21f-4d2e-915b-a5bb52a1929f_f70c1877-6db5-4024-b260-69f50314b40c,3c138178-0520-4950-af40-23fce8501056_fd27af60-7b68-4549-bf35-022eacd74f93,d448c7f9-cd47-4456-8a32-0d0e8e024c14_ad38bbd2-e41c-4509-9dc3-c5c708a36606,31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,73c9e29c-885e-490e-95f3-087227443758_101e55f2-2da0-528b-96fd-21687e242c1c,91ed532a-08f2-4fdd-b5e9-28bb2afd1336_451550a8-d9e6-4396-9f9a-97e752111228
id,,,,,,,,,,,,,,,,,,,,,
d21da01e-f96f-4db5-9746-7b70db8a1f2c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83bafcf1-2f2e-4a32-89a0-f1f16ca10626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
09db39be-d9a6-4fc3-8d25-1f80b23e9131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38a622c6-f086-4763-a952-7c6b3b1c42ba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e9eb024b-6fba-4dce-b280-9415b2ba4695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d0aaa60f-40d9-4c12-a486-954333aa549f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cc415ec5-ef9b-4239-b423-32906818e457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
